# Analyzing CIA Factbook Data Using SQL 



** Introduction**

In this project, we'll work with data from the  [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:
- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

In this project, we'll use SQL in Jupyter Notebook to explore and analyze data from this database. The SQLite factbook.db database can be downloaded from the repository.

We'll use the following code to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

To run SQL queries in this project we add %%sql on its own line to the start of our query.

**Query to return information on the tables in the database**

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


**Executing the first 5 rows of the facts table in the database**

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:
- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

**Summary statistics to identify outlier countries**

In [5]:
%%sql
SELECT MIN(population) min_pop, MAX(population) max_pop, 
       MIN(population_growth) min_pop_growth,
       MAX(population_growth) max_pop_growth
    FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics:
- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

 **Using subqueries to zoom in on just these countries without using the specific values**
 
 
 Executing a query that returns the countrie(s) with the minimum population.

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population IN (SELECT MIN(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Executing a query that returns the countrie(s) with the maximum population.

In [7]:
%%sql
SELECT *
  FROM facts
 WHERE population IN (SELECT MAX(population)
                        FROM facts
                     );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

**Recomputing the summary statistics we found earlier while excluding the row for the whole world**

In [8]:
%%sql
SELECT MIN(population) min_pop, MAX(population) max_pop, 
       MIN(population_growth) min_pop_growth,
       MAX(population_growth) max_pop_growth
    FROM facts
 WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


From the result summary, we found out that a country maximum popuplation approaches 1.4 billion.

** Exploring the average values for population and area**

In [9]:
%%sql
SELECT AVG(population) pop_avg, AVG(area) area_avg
  FROM facts
WHERE name <> 'World';    


Done.


pop_avg,area_avg
32242666.56846473,555093.546184739


From the table above, we could see that the average population is approximately 32 million and average area is also 555 thousand square kilometers.

**Identifying countries that are densely populated**

We'll identify countries that have:
- Above average values for population.
- Below average values for area.


In [12]:
%%sql
SELECT *
   FROM facts
WHERE population > (SELECT (AVG(population))
                        FROM facts
                     WHERE name <> 'World')
AND

    area < (SELECT (AVG(area))
                        FROM facts
                     WHERE name <> 'World');


Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


The table above depicts all the countries that met both  conditions for population above average and area below average.

**Exploring the country with most people and highest growth rate**

In [13]:
%%sql
SELECT name, 
        MAX(population),
        MAX(population_growth)
   FROM facts;

Done.


name,MAX(population),MAX(population_growth)
South Sudan,7256490011,4.02


The result shows Sudan to be the country with most people as well as highest growth rate.

**Exploring countries with the highest ratios of water to land**

In [25]:
%%sql
SELECT name,
CAST(area_water AS Float)/CAST(area_land AS Float) 'water:land'
   FROM facts;

Done.


name,water:land
Afghanistan,0.0
Albania,0.04927366961092051
Algeria,0.0
Andorra,0.0
Angola,0.0
Antigua and Barbuda,0.0
Argentina,0.015971849204696183
Armenia,0.054604120129064286
Australia,0.007669578121135597
Austria,0.017296379404451452


**Exploring countries that have more water than land**

In [14]:
%%sql
SELECT name
 FROM facts
WHERE (area_water > area_land);    

Done.


name
British Indian Ocean Territory
Virgin Islands


British Indian Ocean Territory and Virgin Islands are the countries with more water than land respectively.

**Exploring countries with increase population growth next year**

In [15]:
%%sql
SELECT name, population_growth
 FROM facts
ORDER BY population_growth DESC
LIMIT 10;    

Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


The above countries will add the most people to their population next year.

**Exploring countries that have a higher death rate than birth rate **

In [16]:
%%sql
SELECT name
 FROM facts
WHERE (death_rate > birth_rate);    

Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


**Exploring countries that have the highest population/area ratio **

In [17]:
%%sql
SELECT name,
CAST((population) AS Float)/CAST((area) AS Float) highest_population_area
 FROM facts
LIMIT 5;    

Done.


name,highest_population_area
Afghanistan,49.92769728470018
Albania,105.37352163628775
Algeria,16.602210735760103
Andorra,182.86324786324786
Angola,15.741840859870058


**Conclusion**

In conclusion, in this project we were able to answer most of the questions posed in the problem statement of our work with the use of SQL in Jupyter notebook.